In [ ]:
import pandas as pd
import itertools
id = [1, 2, 3, 4, 5, 6]
origin = ['SFO', 'SFO', 'SFO', 'DFW', 'DFW', 'JFK']
destination = ['JFK', 'DFW', 'MCO', 'MCO', 'JFK', 'LHR']
cost = [500, 200, 400, 100, 200, 1000]
test_df=pd.DataFrame({'id':id, 'origin':origin, 'destination':destination, 'cost':cost})
da_flights=pd.DataFrame({'id':id, 'origin':origin, 'destination':destination, 'cost':cost})
import pandas as pd
import itertools

df = pd.DataFrame(list(itertools.product(da_flights['origin'].unique(), da_flights['destination'].unique())),
                  columns=['origin', 'destination'])
df = df[df['origin'] != df['destination']]

connections_1 = pd.merge(da_flights, da_flights,
                         how='left', left_on='destination', right_on='origin', suffixes=['_0', '_1'])
connections_2 = pd.merge(connections_1, da_flights[['origin', 'destination', 'cost']],
                         how='left', left_on='destination_1', right_on='origin', suffixes=['', '_2']).fillna(0)
connections_2.columns = ['id_0', 'origin_0', 'destination_0', 'cost_0', 'id_1', 'origin_1', 'destination_1',
                         'cost_1', 'origin_2', 'destination_2', 'cost_2']
connections_2['cost_v1'] = connections_2['cost_0'] + connections_2['cost_1']
connections_2['cost_v2'] = connections_2['cost_0'] + connections_2['cost_1'] + connections_2['cost_2']

result = pd.merge(df, da_flights[['origin', 'destination', 'cost']], how='left', on=['origin', 'destination'])

result = pd.merge(result, connections_2[['origin_0', 'destination_1', 'cost_v1']], how='left',
                  left_on=['origin', 'destination'], right_on=['origin_0', 'destination_1'])

result = pd.merge(result, connections_2[['origin_0', 'destination_2', 'cost_v2']], how='left',
                  left_on=['origin', 'destination'], right_on=['origin_0', 'destination_2'])
result['min_price'] = result[['cost', 'cost_v1', 'cost_v2']].min(axis=1)
result[~result['min_price'].isna()][['origin', 'destination', 'min_price']]


In [77]:
all_airports = set()
for i in  range(len(da_flights)):
    all_airports.add(da_flights['origin'][i])
    all_airports.add(da_flights['destination'][i])
all_airports = list(all_airports)
all_airports.sort()
#create a dictionary of all possible routes from origin to destination with cost
all_routes = {}
for i in range(len(all_airports)):
    for j in range(len(all_airports)):
        if i != j:
            all_routes[(all_airports[i], all_airports[j])] = []
for i in range(len(da_flights)):
    all_routes[(da_flights['origin'][i], da_flights['destination'][i])].append(da_flights['cost'][i])
#find the minimum cost for each route
min_cost = {}
for i in range(len(all_airports)):
    for j in range(len(all_airports)):
        if i != j:
            if len(all_routes[(all_airports[i], all_airports[j])]) > 0:
                min_cost[(all_airports[i], all_airports[j])] = min(all_routes[(all_airports[i], all_airports[j])])
            else:
                min_cost[(all_airports[i], all_airports[j])] = 0
#find the minimum cost for each route with one stop
min_cost_1 = {}
for i in range(len(all_airports)):
    for j in range(len(all_airports)):
        if i != j:
            min_cost_1[(all_airports[i], all_airports[j])] = 0
            for k in range(len(all_airports)):
                if i != k and j != k:
                    if min_cost[(all_airports[i], all_airports[k])] > 0 and min_cost[(all_airports[k], all_airports[j])] > 0:
                        if min_cost_1[(all_airports[i], all_airports[j])] == 0:
                            min_cost_1[(all_airports[i], all_airports[j])] = min_cost[(all_airports[i], all_airports[k])] + min_cost[(all_airports[k], all_airports[j])]
                        else:
                            min_cost_1[(all_airports[i], all_airports[j])] = min(min_cost_1[(all_airports[i], all_airports[j])], min_cost[(all_airports[i], all_airports[k])] + min_cost[(all_airports[k], all_airports[j])])
#find the minimum cost for each route with two stops
min_cost_2 = {}
for i in range(len(all_airports)):
    for j in range(len(all_airports)):
        if i != j:
            min_cost_2[(all_airports[i], all_airports[j])] = 0
            for k in range(len(all_airports)):
                if i != k and j != k:
                    for l in range(len(all_airports)):
                        if i != l and j != l and k != l:
                            if min_cost[(all_airports[i], all_airports[k])] > 0 and min_cost[(all_airports[k], all_airports[l])] > 0 and min_cost[(all_airports[l], all_airports[j])] > 0:
                                if min_cost_2[(all_airports[i], all_airports[j])] == 0:
                                    min_cost_2[(all_airports[i], all_airports[j])] = min_cost[(all_airports[i], all_airports[k])] + min_cost[(all_airports[k], all_airports[l])] + min_cost[(all_airports[l], all_airports[j])]
                                else:
                                    min_cost_2[(all_airports[i], all_airports[j])] = min(min_cost_2[(all_airports[i], all_airports[j])], min_cost[(all_airports[i], all_airports[k])] + min_cost[(all_airports[k], all_airports[l])] + min_cost[(all_airports[l], all_airports[j])])
#get the minimum cost for each route from min_cost, min_cost_1, and min_cost_2 for each route and store it in a dataframe with origin, destination, and min_cost columns but the result must be greater than 0
print(min_cost)
print('----------------')
print(min_cost_1)
print('----------------')
print(min_cost_2)
#delete the routes with cost 0 from min_cost, min_cost_1, and min_cost_2
for i in range(len(all_airports)):
    for j in range(len(all_airports)):
        if i != j:
            if min_cost[(all_airports[i], all_airports[j])] == 0:
                del min_cost[(all_airports[i], all_airports[j])]
            if min_cost_1[(all_airports[i], all_airports[j])] == 0:
                del min_cost_1[(all_airports[i], all_airports[j])]
            if min_cost_2[(all_airports[i], all_airports[j])] == 0:
                del min_cost_2[(all_airports[i], all_airports[j])]
#combine min_cost, min_cost_1, and min_cost_2 into one dictionary
min_cost.update(min_cost_1)
min_cost.update(min_cost_2)
#convert the dictionary into a dataframe
min_cost

{('DFW', 'JFK'): 200, ('DFW', 'LHR'): 0, ('DFW', 'MCO'): 100, ('DFW', 'SFO'): 0, ('JFK', 'DFW'): 0, ('JFK', 'LHR'): 1000, ('JFK', 'MCO'): 0, ('JFK', 'SFO'): 0, ('LHR', 'DFW'): 0, ('LHR', 'JFK'): 0, ('LHR', 'MCO'): 0, ('LHR', 'SFO'): 0, ('MCO', 'DFW'): 0, ('MCO', 'JFK'): 0, ('MCO', 'LHR'): 0, ('MCO', 'SFO'): 0, ('SFO', 'DFW'): 200, ('SFO', 'JFK'): 500, ('SFO', 'LHR'): 0, ('SFO', 'MCO'): 400}
----------------
{('DFW', 'JFK'): 0, ('DFW', 'LHR'): 1200, ('DFW', 'MCO'): 0, ('DFW', 'SFO'): 0, ('JFK', 'DFW'): 0, ('JFK', 'LHR'): 0, ('JFK', 'MCO'): 0, ('JFK', 'SFO'): 0, ('LHR', 'DFW'): 0, ('LHR', 'JFK'): 0, ('LHR', 'MCO'): 0, ('LHR', 'SFO'): 0, ('MCO', 'DFW'): 0, ('MCO', 'JFK'): 0, ('MCO', 'LHR'): 0, ('MCO', 'SFO'): 0, ('SFO', 'DFW'): 0, ('SFO', 'JFK'): 400, ('SFO', 'LHR'): 1500, ('SFO', 'MCO'): 300}
----------------
{('DFW', 'JFK'): 0, ('DFW', 'LHR'): 0, ('DFW', 'MCO'): 0, ('DFW', 'SFO'): 0, ('JFK', 'DFW'): 0, ('JFK', 'LHR'): 0, ('JFK', 'MCO'): 0, ('JFK', 'SFO'): 0, ('LHR', 'DFW'): 0, ('LHR', '

{('DFW', 'JFK'): 200,
 ('DFW', 'MCO'): 100,
 ('JFK', 'LHR'): 1000,
 ('SFO', 'DFW'): 200,
 ('SFO', 'JFK'): 400,
 ('SFO', 'MCO'): 300,
 ('DFW', 'LHR'): 1200,
 ('SFO', 'LHR'): 1400}